# Variables

In [1]:
YOUR_NAME = 'sara'

AWS_PROFILE = 'cities'

'''
# List of cities to process
cities = ["Belo Horizonte", "Campinas"]#, "Bogota", "Nairobi", "Bamako", 
        #"Lagos", "Accra", "Abidjan", "Mogadishu", "Cape Town", 
        #"Maputo", "Luanda"]

test_cities = ["Belo Horizonte"]
#cities = test_cities

cities = [city.replace(' ', '_') for city in cities]

search_buffer_files = fs.ls(SEARCH_BUFFER_PATH)

cities 

number_of_cities = len(cities)

print(f'City count: {number_of_cities}')
'''
grid_size = 200


# Setup

In [2]:
%load_ext autoreload

In [3]:
MAIN_PATH = "s3://wri-cities-sandbox/identifyingLandSubdivisions/data"
INPUT_PATH = f'{MAIN_PATH}/input'
CITY_INFO_PATH = f'{INPUT_PATH}/city_info'
EXTENTS_PATH = f'{CITY_INFO_PATH}/extents'
BUILDINGS_PATH = f'{INPUT_PATH}/buildings'
BLOCKS_PATH = f'{INPUT_PATH}/blocks'
ROADS_PATH = f'{INPUT_PATH}/roads'
INTERSECTIONS_PATH = f'{INPUT_PATH}/intersections'
GRIDS_PATH = f'{INPUT_PATH}/city_info/grids'
SEARCH_BUFFER_PATH = f'{INPUT_PATH}/city_info/search_buffers'
OUTPUT_PATH = f'{MAIN_PATH}/output'
OUTPUT_PATH_CSV = f'{OUTPUT_PATH}/csv'
OUTPUT_PATH_RASTER = f'{OUTPUT_PATH}/raster'
OUTPUT_PATH_PNG = f'{OUTPUT_PATH}/png'
OUTPUT_PATH_RAW = f'{OUTPUT_PATH}/raw_results'

In [4]:
# Check s3 connection using AWS_PROFILE=CitiesUserPermissionSet profile 
import boto3

session = boto3.Session(profile_name=AWS_PROFILE)
s3 = session.client('s3')

# export CitiesUserPermissionSet profile to use in the next cells
import os
os.environ['AWS_PROFILE'] = AWS_PROFILE


s3.list_buckets()

{'ResponseMetadata': {'RequestId': 'WB1BCRT3WKRBNC18',
  'HostId': 'elmxKwiBXoK5CqSlLGUQrJwO7ZxC0N8DZsz4E+bnX/PuEkPv8cZmz1x7IRn1cBZP2FpJv+FxOPs=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'elmxKwiBXoK5CqSlLGUQrJwO7ZxC0N8DZsz4E+bnX/PuEkPv8cZmz1x7IRn1cBZP2FpJv+FxOPs=',
   'x-amz-request-id': 'WB1BCRT3WKRBNC18',
   'date': 'Tue, 01 Apr 2025 21:45:34 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'aft-sandbox-540362055257',
   'CreationDate': datetime.datetime(2022, 9, 13, 15, 12, 20, tzinfo=tzutc())},
  {'Name': 'amplify-citiesindicatorsapi-dev-10508-deployment',
   'CreationDate': datetime.datetime(2023, 8, 30, 5, 5, 13, tzinfo=tzutc())},
  {'Name': 'cities-dev-sandbox',
   'CreationDate': datetime.datetime(2025, 2, 7, 23, 18, 12, tzinfo=tzutc())},
  {'Name': 'cities-heat',
   'CreationDate': datetime.datetime(2023, 6, 1, 13, 22, 1, tzinfo=tzutc())},
  {'Name': 'era5-bra

In [5]:
import coiled

cluster = coiled.Cluster(
    workspace="wri-cities-data",
    name=f'ils-{YOUR_NAME}',
    region="us-west-2",
    arm=True,
    worker_vm_types="r8g.xlarge",
    spot_policy="spot",
    n_workers=4,
    package_sync_ignore=["pyspark", "pypandoc"]
)
client = cluster.get_client()

print(f"Started a new Dask client on Coiled. Dashboard is available at {client.dashboard_link}")


[2025-04-01 16:45:35,867][INFO    ][coiled] Fetching latest package priorities...
[2025-04-01 16:45:35,869][INFO    ][coiled.package_sync] Resolving your local subdivisions2 Python environment...
[2025-04-01 16:45:36,815][INFO    ][coiled.package_sync] Scanning 444 conda packages...
[2025-04-01 16:45:36,825][INFO    ][coiled.package_sync] Scanning 259 python packages...
[2025-04-01 16:45:37,724][INFO    ][coiled] Running pip check...
[2025-04-01 16:45:39,290][INFO    ][coiled] Validating environment...
[2025-04-01 16:45:41,833][INFO    ][coiled] Creating wheel for ~/Documents/Identifying Land Subdivisions/identifyingLandSubdivisions...
[2025-04-01 16:45:42,032][WARNING ][coiled.package_sync] Package - libopenvino-intel-cpu-plugin, libopenvino-intel-cpu-plugin~=2025.0.0 has no install candidate for Python 3.12 linux-aarch64 on conda-forge
[2025-04-01 16:45:42,033][INFO    ][coiled] Uploading coiled_local_identifyingLandSubdivisions...
[2025-04-01 16:45:42,974][INFO    ][coiled] Requesti

Started a new Dask client on Coiled. Dashboard is available at https://cluster-qrnhy.dask.host/C1NczZBVgZrXV8jZ/status


# RUN

In [6]:
import s3fs
import fsspec
import traceback

import os


fs = s3fs.S3FileSystem(anon=False)
search_buffer_files = fs.ls(SEARCH_BUFFER_PATH)

cities = [x.split('/')[-1] for x in search_buffer_files]
len(cities)

406

In [7]:
'''
THIS IS PRE-PROCESSING
'''

import dask_geopandas as dgpd
import pandas as pd
from dask import delayed, compute, visualize
import geopandas as gpd
from dask.diagnostics import ProgressBar
from citywide_calculation import get_utm_crs
from metrics_calculation import calculate_minimum_distance_to_roads_option_B
from shapely.geometry import MultiLineString, LineString, Point
from shapely.ops import polygonize, nearest_points
#from shapely.geometry import Polygon, LineString, Point, MultiPolygon, MultiLineString, GeometryCollection
from scipy.optimize import fminbound, minimize
from shapely.ops import unary_union, polygonize
import geopandas as gpd
from polylabel import polylabel
from shapely.geometry import mapping
from shapely.geometry import mapping, Point
from polylabel import polylabel


@delayed
def get_epsg(city_name):
    search_buffer = f'{SEARCH_BUFFER_PATH}/{city_name}/{city_name}_search_buffer.geoparquet'
    extent = dgpd.read_parquet(search_buffer)
    geometry = extent.geometry[0].compute()
    epsg = get_utm_crs(geometry)
    print(f'{city_name} EPSG: {epsg}')
    return epsg

def load_dataset(path, epsg=None):
    dataset = dgpd.read_parquet(path, npartitions=4)
    
    # Only assign if the file has no CRS
    if epsg:
        if dataset.crs is None:
            dataset = dataset.set_crs("EPSG:4326")  # assume WGS84 if missing
        dataset = dataset.to_crs(epsg)

    return dataset

In [8]:

def to_geojson_dict(geom):
    """
    Convert a Shapely geometry to a GeoJSON-like dict with lists instead of tuples.
    """
    geojson = mapping(geom)
    def recursive_convert(obj):
        if isinstance(obj, tuple):
            return list(obj)
        elif isinstance(obj, list):
            return [recursive_convert(item) for item in obj]
        elif isinstance(obj, dict):
            return {k: recursive_convert(v) for k, v in obj.items()}
        else:
            return obj
    return recursive_convert(geojson)

def compute_largest_inscribed_circle(geom):
    """
    Compute the largest inscribed circle for a given polygon or multipolygon.

    Parameters:
      geom (shapely.geometry): A Polygon or MultiPolygon.
    
    Returns:
      tuple: (optimal_point, max_radius) where optimal_point is a shapely Point and max_radius is a float.
    """
    if geom is None or geom.is_empty:
        return None, None

    if geom.geom_type == 'Polygon':
        geojson_poly = to_geojson_dict(geom)
        # Pass in the coordinates list instead of the entire dict.
        optimal_coords = polylabel(geojson_poly["coordinates"])
        optimal = Point(optimal_coords)
        radius = geom.boundary.distance(optimal)
        return optimal, radius

    elif geom.geom_type == 'MultiPolygon':
        best_point = None
        best_radius = 0
        for poly in geom.geoms:
            geojson_poly = to_geojson_dict(poly)
            optimal_coords = polylabel(geojson_poly["coordinates"])
            candidate = Point(optimal_coords)
            radius = poly.boundary.distance(candidate)
            if radius > best_radius:
                best_radius = radius
                best_point = candidate
        return best_point, best_radius

    else:
        return None, None

def add_inscribed_circle_info(blocks_gdf):
    """
    Adds two new columns to a blocks GeoDataFrame: 'optimal_point' and 'max_radius'
    which indicate the center and radius of the largest inscribed circle for each block.
    Converts the optimal_point geometries to WKT strings for Parquet compatibility.
    
    Parameters:
      blocks_gdf (GeoDataFrame): A GeoDataFrame with block polygons.
      
    Returns:
      GeoDataFrame: The input GeoDataFrame with two new columns.
    """
    # Apply the computation for each geometry
    results = blocks_gdf.geometry.apply(lambda geom: compute_largest_inscribed_circle(geom))
    
    # Unpack the tuple results into two new columns
    blocks_gdf["optimal_point"] = results.apply(lambda x: x[0])
    blocks_gdf["max_radius"] = results.apply(lambda x: x[1])
    
    # Convert the 'optimal_point' column from Shapely objects to WKT strings
    blocks_gdf["optimal_point"] = blocks_gdf["optimal_point"].apply(
        lambda geom: geom.wkt if geom is not None else None
    )
    
    return blocks_gdf


def get_blocks(roads):
    """
    Create urban blocks from a grid and road network.

    Parameters:
      grid (GeoDataFrame): A GeoDataFrame of grid polygons defining the city extent.
      roads (GeoDataFrame): A GeoDataFrame of road line geometries.
    
    Returns:
      GeoDataFrame: A GeoDataFrame of block polygons.
    """
    # Merge all road geometries into a single geometry
    roads_union = unary_union(roads.geometry)
    
    # Polygonize the road network to generate blocks.
    # The polygonize function returns an iterator of Polygons.
    blocks_polygons = list(polygonize(roads_union))
    
    # Create a GeoDataFrame for blocks
    blocks_gdf = gpd.GeoDataFrame(geometry=blocks_polygons, crs=roads.crs)
    
    # Remove any empty geometries resulting from the intersection.
    blocks_gdf = blocks_gdf[~blocks_gdf.is_empty]
    
    return blocks_gdf

@delayed
def produce_blocks(city_name):
    # Construct file paths for the city
    paths = {
        'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
        'buildings': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}.geoparquet',
        'roads': f'{ROADS_PATH}/{city_name}/{city_name}_OSM_roads.geoparquet',
        'intersections': f'{INTERSECTIONS_PATH}/{city_name}/{city_name}_OSM_intersections.geoparquet'
    }
    
    epsg = get_epsg(city_name)
    
    roads = load_dataset(paths['roads'], epsg=epsg)
    
    blocks = get_blocks(roads.compute())

    # Now add the inscribed circle information.
    blocks = add_inscribed_circle_info(blocks)
    
    # Define the output path for the blocks geoparquet
    path_blocks = f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}.geoparquet'

    blocks = blocks.set_crs(epsg.compute())

    # Convert the geometry column to WKT before saving
    #blocks["geometry"] = blocks["geometry"].apply(lambda geom: geom.wkt if geom is not None else None)
    
    # Save the blocks dataset. 
    blocks.to_parquet(path_blocks)
    
    # Optionally, return the output path or any summary info.
    return blocks


import time
'''
start_time = time.time()  

#cities = ['Nairobi','Belo_Horizonte']
cities = ["Belo Horizonte", 'Nairobi'] #"Campinas", "Bogota", "Nairobi", "Bamako", "Lagos", "Accra", "Abidjan", "Cape Town", "Luanda"] #"Maputo","Mogadishu", 
cities = [city.replace(' ', '_') for city in cities]

tasks = [produce_blocks(city) for city in cities]
results = compute(*tasks)

end_time = time.time()  
elapsed_time = end_time - start_time

print(f"Tasks completed in {elapsed_time:.2f} seconds.")'
'''

'\nstart_time = time.time()  \n\n#cities = [\'Nairobi\',\'Belo_Horizonte\']\ncities = ["Belo Horizonte", \'Nairobi\'] #"Campinas", "Bogota", "Nairobi", "Bamako", "Lagos", "Accra", "Abidjan", "Cape Town", "Luanda"] #"Maputo","Mogadishu", \ncities = [city.replace(\' \', \'_\') for city in cities]\n\ntasks = [produce_blocks(city) for city in cities]\nresults = compute(*tasks)\n\nend_time = time.time()  \nelapsed_time = end_time - start_time\n\nprint(f"Tasks completed in {elapsed_time:.2f} seconds.")\'\n'

In [9]:
from scipy.stats import t, sem, entropy
import numpy as np

'''
Auxiliary functions for metric 6
'''

def calculate_standardized_kl_azimuth(buildings_df, bin_width_degrees=5):
    azimuths = buildings_df['azimuth'].to_numpy()
    num_bins = int(90 / bin_width_degrees)
    histogram, _ = np.histogram(azimuths, bins=num_bins, range=(0, 90))
    P = histogram / histogram.sum() if histogram.sum() > 0 else np.ones(num_bins) / num_bins
    Q = np.ones(num_bins) / num_bins
    kl_divergence = entropy(P, Q)
    max_kl_divergence = np.log(num_bins)
    return kl_divergence / max_kl_divergence

def compute_azimuth_partition(df):
    def azimuth(geom):
        if geom is None or geom.is_empty:
            return np.nan
        oriented = geom.minimum_rotated_rectangle
        coords = list(oriented.exterior.coords)
        edge = LineString([coords[0], coords[1]])
        dx, dy = edge.xy[0][1] - edge.xy[0][0], edge.xy[1][1] - edge.xy[1][0]
        angle = np.degrees(np.arctan2(dy, dx)) % 180
        return angle % 90

    df = df.copy()
    df['azimuth'] = df['geometry'].map(azimuth)
    return df

@delayed
def compute_block_kl_metrics(buildings_blocks):
    grouped = buildings_blocks.groupby('block_id')
    kl_data = grouped.apply(lambda g: pd.Series({
        'standardized_kl': calculate_standardized_kl_azimuth(g),
        'n_buildings': len(g),
    })).reset_index()
    return kl_data

def compute_block_grid_weights(blocks, grid):
    """
    Computes the proportional overlap of blocks in each grid cell.
    Returns a Dask DataFrame containing block_id, index_right (grid ID), and area_weight.
    """


    #blocks = blocks.rename_axis(index='block_id').reset_index()
    grid = grid.rename_axis(index='grid_id').reset_index()

    def overlay_partition(blocks_df, grid_df):
        """Computes intersection between blocks and grid."""
        return gpd.overlay(blocks_df, grid_df, how='intersection')

    #meta = blocks._meta.merge(grid._meta, how="outer")

    block_grid_overlap = blocks.map_partitions(overlay_partition, grid)#, meta=meta


    # Step 2: Compute area for each block-grid overlap
    block_grid_overlap = block_grid_overlap.assign(
        overlap_area=block_grid_overlap.map_partitions(lambda df: df.geometry.area, meta=('overlap_area', 'f8'))
    )

    # Step 3: Compute the total area of each grid cell
    grid_areas = grid.assign(grid_area=grid.map_partitions(lambda df: df.geometry.area, meta=('grid_area', 'f8')))


    # Step 4: Merge grid cell areas into block-grid overlap
    block_grid_overlap = block_grid_overlap.merge(grid_areas[['grid_id','grid_area']], left_on='grid_id', right_on='grid_id', how='left')

    # Step 5: Compute area weight as the ratio of overlap to grid cell area
    block_grid_overlap = block_grid_overlap.assign(
        area_weight=block_grid_overlap['overlap_area'] / block_grid_overlap['grid_area']
    )
    block_grid_overlap = block_grid_overlap.map_partitions(
        lambda df: df.assign(
            area_weight=df['area_weight'] / df.groupby(df['grid_id'])['area_weight'].transform('sum')
        ),
        meta=block_grid_overlap._meta  # Preserve original structure
    )

    return block_grid_overlap[['block_id', 'optimal_point', 'max_radius', 'grid_id', 'geometry', 'overlap_area', 'grid_area', 'area_weight']]


def aggregate_m6(kl_df, overlap_df):
    df = overlap_df.merge(kl_df, on='block_id', how='left')
    df = df.dropna(subset=['standardized_kl'])

    # Compute weights
    df['weight'] = df['area_weight'] * df['n_buildings']
    df['weighted_kl'] = df['standardized_kl'] * df['weight']

    # Aggregate directly at the GRID level
    grid_aggregated = df.groupby('grid_id').agg(
        total_weighted_kl=('weighted_kl', 'sum'),
        total_weight=('weight', 'sum')
    )

    # Compute final KL divergence for each grid cell
    grid_aggregated['m6'] = grid_aggregated['total_weighted_kl'] / grid_aggregated['total_weight']

    return grid_aggregated[['m6']]



def building_orientation_metrics(city_name):
    paths = {
        'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
        'blocks': f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}.geoparquet',
        'buildings_with_distances': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances.geoparquet',
    }

    epsg = get_epsg(city_name).compute()
    grid = load_dataset(paths['grid'], epsg=epsg)
    blocks = load_dataset(paths['blocks'], epsg=epsg)
    buildings = load_dataset(paths['buildings_with_distances'], epsg=epsg)

    if 'geom' in grid.columns:
        grid = grid.drop(columns=['geom'])

    blocks['block_id'] = blocks.index

    meta = buildings._meta.copy()
    meta['azimuth'] = 'f8'
    buildings = buildings.map_partitions(compute_azimuth_partition, meta=meta)

    # Fix `sjoin` issues by computing before
    buildings_blocks = dgpd.sjoin(buildings.compute(), blocks.compute(), predicate='intersects')
    buildings_blocks = buildings_blocks[['block_id', 'geometry', 'azimuth']]

    kl_df = compute_block_kl_metrics(buildings_blocks)

    # Keep `block_grid_overlap` lazy
    block_grid_overlap = compute_block_grid_weights(blocks, grid)

    # Aggregate `m6`
    m6_grid = aggregate_m6(kl_df.compute(), block_grid_overlap.compute())
    grid = grid.merge(m6_grid, left_index=True, right_index=True, how='left')
    grid['m6'] = grid['m6'].fillna(0)


    path = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{str(grid_size)}m_grid_{YOUR_NAME}_metric_6.geoparquet'
    grid.to_parquet(path)
    return path

In [10]:
import dask_geopandas as dgpd
import pandas as pd
from dask import delayed, compute, visualize
import geopandas as gpd
from dask.diagnostics import ProgressBar
from citywide_calculation import get_utm_crs
from metrics_calculation import calculate_minimum_distance_to_roads_option_B
from shapely.geometry import MultiLineString, LineString, Point
from shapely.ops import polygonize, nearest_points
#from shapely.geometry import Polygon, LineString, Point, MultiPolygon, MultiLineString, GeometryCollection
from scipy.optimize import fminbound, minimize
import shapely.wkt


In [11]:

@delayed
def block_metrics(city_name, YOUR_NAME, grid_size):
    paths = {
        'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
        'buildings': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}.geoparquet',
        'roads': f'{ROADS_PATH}/{city_name}/{city_name}_OSM_roads.geoparquet',
        'intersections': f'{INTERSECTIONS_PATH}/{city_name}/{city_name}_OSM_intersections.geoparquet',
        'blocks' : f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}.geoparquet'
    }
    # Get EPSG (you may still use a delayed get_epsg, but ensure you compute it if necessary)
    epsg = get_epsg(city_name).compute()
    
    grid = load_dataset(paths['grid'], epsg=epsg)
    blocks = load_dataset(paths['blocks'])

    if 'geom' in grid.columns:
        grid = grid.drop(columns='geom')
    if 'geom' in blocks.columns:
        blocks = blocks.drop(columns='geom')

    # Ensure that the active geometry is set correctly
    grid = grid.set_geometry("geometry")
    blocks = blocks.set_geometry("geometry")

    grid = grid.persist()
    blocks = blocks.persist()

    import matplotlib.pyplot as plt

    grid_sample = grid.head()
    blocks_sample = blocks.head()

    ax = grid_sample.plot(edgecolor='black', facecolor='none', figsize=(8, 8))
    blocks_sample.plot(ax=ax, color='red', alpha=0.5)
    plt.show()


    # Perform the spatial join with dask-geopandas GeoDataFrames
    blocks_grid_joined = dgpd.sjoin(blocks, grid, predicate='intersects')
    average_block_maxradius = blocks_grid_joined.groupby('index_right')['max_radius'].mean().astype(float)
    
    grid['m7'] = grid.index.map(average_block_maxradius).fillna(0).astype(float)
    
    path = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{str(grid_size)}m_grid_{YOUR_NAME}.geoparquet'
    
    # Remove problematic column if present
    if 'geom' in grid.columns:
        grid = grid.drop(columns='geom')
    
    grid.to_parquet(path)
    return path

'''
import time

start_time = time.time()  # Start the timer

#cities = ['Nairobi','Belo_Horizonte']
cities = ["Belo Horizonte", "Campinas", "Bogota", "Nairobi", "Bamako", "Lagos", "Accra", "Abidjan", "Cape Town", "Luanda"] #"Maputo","Mogadishu", 
cities = [city.replace(' ', '_') for city in cities]

tasks = [block_metrics(city,YOUR_NAME,grid_size) for city in cities]
results = compute(*tasks)

end_time = time.time()  # End the timer
elapsed_time = end_time - start_time

results

print(f"Tasks completed in {elapsed_time:.2f} seconds.")'
'''

'\nimport time\n\nstart_time = time.time()  # Start the timer\n\n#cities = [\'Nairobi\',\'Belo_Horizonte\']\ncities = ["Belo Horizonte", "Campinas", "Bogota", "Nairobi", "Bamako", "Lagos", "Accra", "Abidjan", "Cape Town", "Luanda"] #"Maputo","Mogadishu", \ncities = [city.replace(\' \', \'_\') for city in cities]\n\ntasks = [block_metrics(city,YOUR_NAME,grid_size) for city in cities]\nresults = compute(*tasks)\n\nend_time = time.time()  # End the timer\nelapsed_time = end_time - start_time\n\nresults\n\nprint(f"Tasks completed in {elapsed_time:.2f} seconds.")\'\n'

In [12]:
from shapely.geometry import Polygon




def calculate_azimuths(city_name):
    paths = {
        'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
        'blocks': f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}.geoparquet',
        'buildings_with_distances': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances.geoparquet',
        'buildings_with_distances_azimuths': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances_and_azimuths.geoparquet',
        'buildings_to_blocks':f'{BLOCKS_PATH}/{city_name}/{city_name}_buildings_to_blocks_{YOUR_NAME}.geoparquet'
    }
    epsg = get_epsg(city_name).compute()
    buildings = load_dataset(paths['buildings_with_distances'], epsg=epsg)
    meta = buildings._meta.copy()
    meta['azimuth'] = 'f8'
    buildings = buildings.map_partitions(compute_azimuth_partition, meta=meta)
    path = paths['buildings_with_distances_azimuths']
    buildings.to_parquet(path)

    return path


@delayed
def pre_process_block_calculations(city_name, epsilon):
    calculate_azimuths(city_name)


'''
import time
from dask import compute

start_time = time.time()  # Start the timer

cities = ['Nairobi']#,'Belo_Horizonte','Medellin','Bogota','Campinas','Luanda','Lagos','Cape_Town'
cities = [city.replace(' ', '_') for city in cities]

tasks = [pre_process_block_calculations(city_name, epsilon=0.001) for city_name in cities]
results = compute(*tasks)  

end_time = time.time()  # End the timer
elapsed_time = end_time - start_time

print(f"Tasks completed in {elapsed_time:.2f} seconds.")'
'''


'\nimport time\nfrom dask import compute\n\nstart_time = time.time()  # Start the timer\n\ncities = [\'Nairobi\']#,\'Belo_Horizonte\',\'Medellin\',\'Bogota\',\'Campinas\',\'Luanda\',\'Lagos\',\'Cape_Town\'\ncities = [city.replace(\' \', \'_\') for city in cities]\n\ntasks = [pre_process_block_calculations(city_name, epsilon=0.001) for city_name in cities]\nresults = compute(*tasks)  \n\nend_time = time.time()  # End the timer\nelapsed_time = end_time - start_time\n\nprint(f"Tasks completed in {elapsed_time:.2f} seconds.")\'\n'

In [15]:

def get_internal_buffer_with_target_area(geom, target_area, tolerance=1e-6, max_iter=100):
    """
    Iteratively finds an internal buffer that results in the target area.

    Parameters:
    - geom: Shapely Polygon geometry.
    - target_area: Desired area for the internal buffer.
    - tolerance: Error tolerance for area difference.
    - max_iter: Maximum iterations to refine buffer.

    Returns:
    - Buffered geometry (Polygon) or the original geometry if buffering is not possible.
    """
    if geom.is_empty or geom.area <= target_area:
        return geom  # Return original if no valid buffer can be made

    buffer_dist = -0.1 * (geom.area ** 0.5)  # Start with a fraction of block size
    iteration = 0

    while iteration < max_iter:
        buffered_geom = geom.buffer(buffer_dist)

        if buffered_geom.is_empty:
            return geom  # If buffering fails, return original geometry

        new_area = buffered_geom.area
        area_diff = abs(new_area - target_area)

        if area_diff < tolerance:
            return buffered_geom  # Found a good enough buffer

        # Adjust buffer distance using a binary search-like approach
        if new_area > target_area:
            buffer_dist *= 1.1  # Increase buffer distance
        else:
            buffer_dist *= 0.9  # Decrease buffer distance

        iteration += 1

    return buffered_geom  # Return best-found buffer



def clip_group(df_group, buffer_type):
    # Use the buffer from the specified column.
    buffer_geom = df_group[buffer_type].iloc[0]
    # Use the original building footprint column for intersection.
    clipped = np.vectorize(lambda geom: geom.intersection(buffer_geom))(df_group['geometry'].values)
    return pd.DataFrame({
        'building_id': df_group['building_id'],
        'block_id': df_group['block_id'],
        'clipped_geometry': clipped
    }, index=df_group.index)


def clip_buildings_by_buffer(buildings_blocks_df, buffer_type):
    # Copy the input and reset the index.
    gdf = buildings_blocks_df.copy().reset_index()
    # If an 'id' column exists, rename it; otherwise, create 'building_id' from the index.
    if 'id' in gdf.columns:
        gdf = gdf.rename(columns={'id': 'building_id'})
    else:
        gdf['building_id'] = gdf.index

    if gdf.crs is None or not gdf.crs.is_projected:
        raise ValueError("GeoDataFrame must have a projected CRS for efficient clipping.")

    # Group by block_id and apply the clipping function.
    clipped_series = gdf.groupby('block_id', group_keys=False).apply(clip_group, buffer_type)
    
    # Create a GeoDataFrame from the result.
    clipped_geo = gpd.GeoDataFrame(clipped_series, geometry='clipped_geometry', crs=buildings_blocks_df.crs)
    
    # Merge the clipped geometries back into the original GeoDataFrame.
    gdf = gdf.merge(clipped_geo[['building_id', 'block_id', 'clipped_geometry']], 
                    on=['building_id', 'block_id'], how='left')
    
    gdf_clipped = gpd.GeoDataFrame(gdf.copy(), geometry='clipped_geometry', crs=buildings_blocks_df.crs)
    gdf_clipped['clipped_area'] = gdf_clipped['clipped_geometry'].area
    gdf_clipped['buffer_area'] = gdf_clipped[buffer_type].area
    
    # Aggregate the areas by block.
    clipped_building_area = gdf_clipped.groupby('block_id')['clipped_area'].sum()
    total_buffer_area = gdf_clipped.groupby('block_id')['buffer_area'].sum()
    ratio = clipped_building_area / total_buffer_area
    return ratio


@delayed
def compute_m6_m7_m8(city_name, YOUR_NAME, grid_size):
    """
    Computes:
    - M6: KL divergence (building orientation)
    - M7: Average block width
    - M8: Building density ratio (inner vs. outer buffer)
    """

    epsilon = 0.001
    paths = {
        'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
        'blocks': f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}.geoparquet',
        'buildings_with_distances': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances.geoparquet',
        'buildings_with_distances_azimuths': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances_and_azimuths.geoparquet',
        'buildings_to_blocks':f'{BLOCKS_PATH}/{city_name}/{city_name}_buildings_to_blocks_{YOUR_NAME}.geoparquet'
    }

    epsg = get_epsg(city_name).compute()
    grid = load_dataset(paths['grid'], epsg=epsg)
    blocks = load_dataset(paths['blocks'], epsg=epsg).persist()
    buildings = load_dataset(paths['buildings_with_distances_azimuths'], epsg=epsg).persist()
    buildings['azimuth'] = buildings['azimuth'].map_partitions(pd.to_numeric, errors='coerce')


    if 'geom' in grid.columns:
        grid = grid.drop(columns=['geom'])
    
    blocks['block_id'] = blocks.index
    blocks['epsilon_buffer'] = blocks['geometry'].buffer(-(1.- epsilon) * blocks['max_radius'])
    blocks['width_buffer'] = blocks['geometry'].buffer(-0.2 * blocks['max_radius'])

    buildings_blocks = dgpd.sjoin(buildings, blocks, predicate='intersects').persist() #,how='right'
    buildings_blocks = buildings_blocks[['block_id', 'geometry', 'epsilon_buffer','width_buffer','azimuth']]
    buildings_blocks = buildings_blocks.set_index('block_id').repartition(npartitions=4)

    block_grid_overlap = compute_block_grid_weights(blocks, grid)
    block_grid_overlap = block_grid_overlap.compute()

    # Metric 6
    kl_df = compute_block_kl_metrics(buildings_blocks)
    m6_grid = aggregate_m6(kl_df.compute(), block_grid_overlap)
 
    # Metric 7
    block_grid_overlap['weighted_max_radius'] = (
        block_grid_overlap['max_radius'] * block_grid_overlap['area_weight']
    )

    grid_m7 = block_grid_overlap.groupby('grid_id').agg(
        total_weighted_max_radius=('weighted_max_radius', 'sum'),
        total_weight=('area_weight', 'sum')
    )
    grid_m7['m7'] = grid_m7['total_weighted_max_radius'] / grid_m7['total_weight']

    # Metric 8
    width_buffer_ratios = buildings_blocks.map_partitions(clip_buildings_by_buffer, buffer_type='width_buffer')
    epsilon_buffer_ratios = buildings_blocks.map_partitions(clip_buildings_by_buffer, buffer_type='epsilon_buffer')
    clipped_buildings_area_to_buffer_ratio = epsilon_buffer_ratios / width_buffer_ratios
    clipped_buildings_area_to_buffer_ratio = clipped_buildings_area_to_buffer_ratio.replace([np.inf, -np.inf], np.nan).fillna(999)
    ratio_df = clipped_buildings_area_to_buffer_ratio.to_frame(name='m8')
    blocks_with_m8 = blocks.merge(ratio_df, left_on='block_id', right_index=True, how='left').compute()
    block_grid_overlap = block_grid_overlap.merge(blocks_with_m8, how='left',left_on='block_id',right_index=True)
    block_grid_overlap['weighted_m8'] = (
        block_grid_overlap['m8'] * block_grid_overlap['area_weight']
    )
    grid_m8 = block_grid_overlap.groupby('grid_id').agg(
        total_weighted_m8=('weighted_m8', 'sum'),
        total_weight=('area_weight', 'sum')
    )
    grid_m8['m8'] = grid_m8['total_weighted_m8'] / grid_m8['total_weight']

    # Merge all metrics
    grid = grid.merge(m6_grid, left_index=True, right_index=True, how='left')
    grid = grid.merge(grid_m7[['m7']], left_index=True, right_index=True, how='left')
    grid = grid.merge(grid_m8[['m8']], left_index=True, right_index=True, how='left')

    '''
    # Fill NaNs
    '''

    grid['m6'] = grid['m6'].fillna(0)
    grid['m7'] = grid['m7'].fillna(0)
    grid['m8'] = grid['m8'].fillna(-999.)
    
    # Save Output
    grid = grid.compute()  
    path = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{str(grid_size)}m_grid_{YOUR_NAME}_metrics_6_7_8.geoparquet'
    grid.to_parquet(path)
    
    #path = f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_with_m8_{YOUR_NAME}.geoparquet'#f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{str(grid_size)}m_grid_{YOUR_NAME}_metrics_6_7_8.geoparquet'
    #blocks_with_m8.to_parquet(path)
    return  path



import time
from dask import compute

start_time = time.time()  # Start the timer

cities = ['Nairobi']
cities = [city.replace(' ', '_') for city in cities]

tasks = [compute_m6_m7_m8(city_name, YOUR_NAME, grid_size) for city_name in cities]
results = compute(*tasks)  

end_time = time.time()  # End the timer
elapsed_time = end_time - start_time

print(f"Tasks completed in {elapsed_time:.2f} seconds.")

Tasks completed in 95.63 seconds.


In [20]:
city_name = 'Nairobi'
blocks_with_m8_0 = gpd.read_parquet(f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}_with_m8.geoparquet/part.0.parquet')
blocks_with_m8_1 = gpd.read_parquet(f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}_with_m8.geoparquet/part.1.parquet')
blocks_with_m8_2 = gpd.read_parquet(f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}_with_m8.geoparquet/part.2.parquet')
blocks_with_m8_3 = gpd.read_parquet(f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}_with_m8.geoparquet/part.3.parquet')

In [33]:
blocks_with_m8 = results[0]
blocks_with_m8.to_parquet('blocks_with_m8_1_minus_epsilon_0.2_max_width.geoparquet')

In [ ]:
blocks_with_m8#[blocks_with_m8.block_id==870]

,geometry,optimal_point,max_radius,block_id,epsilon_buffer,width_buffer,m8
206,"POLYGON ((257194.773 9860138.634, 257242.745 9...",POINT (257564.3624909374 9859864.43161744),98.079789,870,"POLYGON ((257194.725 9860138.548, 257242.697 9...","MULTIPOLYGON (((256963.872 9860172.538, 256975...",0.876002


In [49]:
400462898.4840467/1452820667.2693443#blocks_with_m8.epsilon_buffer.area.sum() #334037.77896078693, 92553.75989874889, 400462898.4840467, 1452820667.2693443

0.27564510025641265

In [13]:
city_name = 'Nairobi'
epsilon = 0.001

paths = {
    'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
    'blocks': f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}.geoparquet',
    'buildings_with_distances': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances.geoparquet',
    'buildings_with_distances_azimuths': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances_and_azimuths.geoparquet',
    'buildings_to_blocks':f'{BLOCKS_PATH}/{city_name}/{city_name}_buildings_to_blocks_{YOUR_NAME}.geoparquet'
}

epsg = get_epsg(city_name).compute()
blocks = load_dataset(paths['blocks'], epsg=epsg)

blocks['block_id'] = blocks.index
blocks['epsilon_buffer'] = blocks['geometry'].buffer(-epsilon * blocks['max_radius'])
blocks['width_buffer'] = blocks['geometry'].buffer(-0.5 * blocks['max_radius'])

blocks = blocks.compute()


In [14]:
buildings = load_dataset(paths['buildings_with_distances_azimuths'], epsg=epsg).compute()

In [15]:
# After the spatial join:
buildings_blocks = dgpd.sjoin(buildings, blocks, predicate='intersects').persist()
#unique_blocks = buildings_blocks['block_id'].nunique().compute()
#print("Unique block_ids in spatial join:", unique_blocks)
#print("Total rows in spatial join:", len(buildings_blocks)) #Total rows in spatial join: 1570337
#print("Unique block_ids in blocks:", blocks['block_id'].nunique())



/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/distributed/client.py:3370: UserWarning: Sending large graph of size 422.07 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


In [17]:
buildings_blocks[buildings_blocks.block_id==3419].compute()

,geometry,distance_to_nearest_road,azimuth,index_right,optimal_point,max_radius,block_id,epsilon_buffer,width_buffer
id,,,,,,,,,
08b7a6e54a49bfff020004fa46fc966a,"POLYGON ((248826.021 9854976.159, 248830.132 9...",16.046173476016552,55.99000353220637,3419,POINT (249387.8739787983 9854904.394042227),305.13438,3419,"POLYGON ((248416.393 9855128.22, 248412.677 98...","POLYGON ((248646.698 9855360.179, 248701.545 9..."
08b7a6e559800fff020060ce7ca2f937,"POLYGON ((249449.718 9855090.983, 249447.928 9...",63.42266999571859,20.756710849882438,3419,POINT (249387.8739787983 9854904.394042227),305.13438,3419,"POLYGON ((248416.393 9855128.22, 248412.677 98...","POLYGON ((248646.698 9855360.179, 248701.545 9..."
08b7a6e559801fff0200f71c799ec630,"POLYGON ((249401.002 9855071.818, 249404.685 9...",102.53707652806955,21.29404677404159,3419,POINT (249387.8739787983 9854904.394042227),305.13438,3419,"POLYGON ((248416.393 9855128.22, 248412.677 98...","POLYGON ((248646.698 9855360.179, 248701.545 9..."
08b7a6e559804fff0200a57a16b4de8b,"POLYGON ((249509.672 9855015.146, 249505.265 9...",115.24224541418795,25.727975833800414,3419,POINT (249387.8739787983 9854904.394042227),305.13438,3419,"POLYGON ((248416.393 9855128.22, 248412.677 98...","POLYGON ((248646.698 9855360.179, 248701.545 9..."
08b7a6e559808fff02006616fff7aaea,"POLYGON ((249328.956 9855125.221, 249328.656 9...",104.09451773638028,7.0611954345631744,3419,POINT (249387.8739787983 9854904.394042227),305.13438,3419,"POLYGON ((248416.393 9855128.22, 248412.677 98...","POLYGON ((248646.698 9855360.179, 248701.545 9..."
...,...,...,...,...,...,...,...,...,...
08b7a6e559b8bfff020071c212c09e33,"POLYGON ((248765.164 9855522.935, 248764.84 98...",12.262088434141244,2.736479721583123,3419,POINT (249387.8739787983 9854904.394042227),305.13438,3419,"POLYGON ((248416.393 9855128.22, 248412.677 98...","POLYGON ((248646.698 9855360.179, 248701.545 9..."
08b7a6e559b8bfff0200cd73bee6b2d7,"POLYGON ((248759.767 9855524.673, 248759.068 9...",10.151673442397184,6.917151097870179,3419,POINT (249387.8739787983 9854904.394042227),305.13438,3419,"POLYGON ((248416.393 9855128.22, 248412.677 98...","POLYGON ((248646.698 9855360.179, 248701.545 9..."
08b7a6e559b8bfff0200f214206249a2,"POLYGON ((248755.036 9855519.079, 248754.806 9...",17.623527537846996,4.348662893852543,3419,POINT (249387.8739787983 9854904.394042227),305.13438,3419,"POLYGON ((248416.393 9855128.22, 248412.677 98...","POLYGON ((248646.698 9855360.179, 248701.545 9..."


In [31]:
buildings_blocks[buildings_blocks.block_id==870].compute()

,geometry,distance_to_nearest_road,azimuth,index_right,optimal_point,max_radius,block_id,epsilon_buffer,width_buffer
id,,,,,,,,,
08b7a6e428000fff02003fb9f2cb81fd,"POLYGON ((257165.429 9860043.549, 257135.451 9...",3.5902859944719845,60.62048095338031,870,POINT (257564.3624909374 9859864.43161744),98.079789,870,"POLYGON ((257194.725 9860138.548, 257242.697 9...","MULTIPOLYGON (((256963.872 9860172.538, 256975..."
08b7a6e428002fff02002e46dd39bd05,"POLYGON ((257175.202 9860056.266, 257163.065 9...",0.7550096264179459,65.78024821038267,870,POINT (257564.3624909374 9859864.43161744),98.079789,870,"POLYGON ((257194.725 9860138.548, 257242.697 9...","MULTIPOLYGON (((256963.872 9860172.538, 256975..."
08b7a6e428002fff02003cbf3f5cf6f5,"POLYGON ((257173.459 9860093.183, 257168.468 9...",3.374771804109713,63.92829133524706,870,POINT (257564.3624909374 9859864.43161744),98.079789,870,"POLYGON ((257194.725 9860138.548, 257242.697 9...","MULTIPOLYGON (((256963.872 9860172.538, 256975..."
08b7a6e428002fff0200488fe729c9ae,"POLYGON ((257159.757 9860063.929, 257142.875 9...",11.992409414033153,66.88616716020653,870,POINT (257564.3624909374 9859864.43161744),98.079789,870,"POLYGON ((257194.725 9860138.548, 257242.697 9...","MULTIPOLYGON (((256963.872 9860172.538, 256975..."
08b7a6e428002fff02004e659355c6b6,"POLYGON ((257162.19 9860079.43, 257151.948 986...",5.38941016386697,60.61944580119112,870,POINT (257564.3624909374 9859864.43161744),98.079789,870,"POLYGON ((257194.725 9860138.548, 257242.697 9...","MULTIPOLYGON (((256963.872 9860172.538, 256975..."
...,...,...,...,...,...,...,...,...,...
08b7a6e428c59fff0200b5b62804e36f,"POLYGON ((257599.018 9859831.84, 257599.533 98...",20.220089944953344,80.91694312126168,870,POINT (257564.3624909374 9859864.43161744),98.079789,870,"POLYGON ((257194.725 9860138.548, 257242.697 9...","MULTIPOLYGON (((256963.872 9860172.538, 256975..."
08b7a6e428c59fff0200d6591ce878bf,"POLYGON ((257609.275 9859827.278, 257580.107 9...",4.479492589701704,80.0712280288098,870,POINT (257564.3624909374 9859864.43161744),98.079789,870,"POLYGON ((257194.725 9860138.548, 257242.697 9...","MULTIPOLYGON (((256963.872 9860172.538, 256975..."
08b7a6e428c59fff0200e9b0cb2f5c9e,"POLYGON ((257624.146 9859824.046, 257609.168 9...",8.881326207001798,80.0712280482829,870,POINT (257564.3624909374 9859864.43161744),98.079789,870,"POLYGON ((257194.725 9860138.548, 257242.697 9...","MULTIPOLYGON (((256963.872 9860172.538, 256975..."


In [29]:
blocks_id_set = set(blocks['block_id'])
bblocks_id_set = set(buildings_blocks['block_id'])

# Get the IDs that are in blocks but NOT in buildings_blocks
missing_block_ids = blocks_id_set - bblocks_id_set

# Filter blocks to those IDs
missing_blocks = blocks[blocks['block_id'].isin(missing_block_ids)]

print(f"Number of blocks missing: {len(missing_blocks)}")
print(missing_blocks.head())


Number of blocks missing: 7323
                                             geometry  \
6   POLYGON ((267359.786 9850914.292, 267281.122 9...   
16  POLYGON ((255229.229 9856779.139, 255238.28 98...   
20  POLYGON ((255758.391 9857820.393, 255745.934 9...   
22  POLYGON ((255678.593 9857700.421, 255672.491 9...   
27  POLYGON ((256642.416 9857463.434, 256733.748 9...   

                                   optimal_point  max_radius  block_id  \
6   POINT (266979.35204538767 9850991.086018158)   98.278196         6   
16  POINT (255238.51091183256 9856754.831162918)    5.679321        16   
20  POINT (255694.80285892845 9857718.934448328)   23.049142        20   
22  POINT (255537.24405270943 9857593.153526023)    8.997523        22   
27   POINT (257097.0789480702 9857679.317723105)    7.314309        27   

                                       epsilon_buffer  \
6   POLYGON ((267359.757 9850914.386, 267281.102 9...   
16  POLYGON ((255229.239 9856779.116, 255238.275 9...   
20  POLYGO

In [ ]:
buildings_blocks

In [ ]:
buildings_blocks = dgpd.sjoin(buildings, blocks, predicate='intersects')
buildings_blocks = buildings_blocks[['block_id', 'geometry', 'epsilon_buffer','width_buffer','azimuth']]


In [17]:
def clip_group_area_temporary(df_group, buffer_type):
    """
    For each group (block), compute the intersection of the building footprints
    with the specified buffer and return the area of the intersection (clipped area)
    along with the area of the buffer.
    """
    # Get the buffer geometry for this group (assumed constant within group)
    buffer_geom = df_group[buffer_type].iloc[0]
    # Compute the intersection of each building footprint with the buffer
    intersections = np.vectorize(lambda geom: geom.intersection(buffer_geom))(df_group['geometry'].values)
    # Compute area for each intersection; if None, use 0
    clipped_areas = [inter.area if inter is not None else 0.0 for inter in intersections]
    # Assume buffer area is constant across the group; compute it once
    buf_area = buffer_geom.area
    # Create a DataFrame of results (only numeric columns)
    return pd.DataFrame({
        'block_id': df_group['block_id'],
        'clipped_area': clipped_areas,
        'buffer_area': [buf_area] * len(df_group)
    }, index=df_group.index)


def clip_buildings_by_buffer_temporary(buildings_blocks_df, buffer_type):
    """
    Instead of returning geometries, this function returns the ratio of the total
    clipped building area to the total buffer area for each block.
    """
    gdf = buildings_blocks_df.copy().reset_index(drop=True)
    # Ensure we have a 'block_id' column and a 'building_id' column
    if 'id' in gdf.columns:
        gdf = gdf.rename(columns={'id': 'building_id'})
    else:
        gdf['building_id'] = gdf.index

    # We assume that gdf already has the proper CRS
    # Use groupby-apply with our new function that computes areas
    area_df = gdf.groupby('block_id', group_keys=False).apply(clip_group_area_temporary, buffer_type)
    
    # Aggregate the results for each block
    total_clipped_area = area_df.groupby('block_id')['clipped_area'].sum()
    total_buffer_area = area_df.groupby('block_id')['buffer_area'].sum()
    
    # Compute the ratio (using division that produces NaN if buffer area is 0)
    ratio = total_clipped_area / total_buffer_area
    return ratio


width_buffer_ratios = clip_buildings_by_buffer_temporary(buildings_blocks, buffer_type='width_buffer')
epsilon_buffer_ratios = clip_buildings_by_buffer_temporary(buildings_blocks, buffer_type='epsilon_buffer')

width_buffer_ratios = width_buffer_ratios.compute()
epsilon_buffer_ratios = epsilon_buffer_ratios.compute()


/var/folders/mt/3n9j2kc92kv4psztx687vtd80000gn/T/ipykernel_6841/1509015195.py:37: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  area_df = gdf.groupby('block_id', group_keys=False).apply(clip_group_area_temporary, buffer_type)
/var/folders/mt/3n9j2kc92kv4psztx687vtd80000gn/T/ipykernel_6841/1509015195.py:37: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  area_df = gdf.groupby('block_id', group_keys=False).apply(clip_group_area_temporary, buffer_type)
/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-pack

RuntimeError: P2P 5d26a429c1e56c52542e2f1cb8ecdcfc failed during transfer phase

,block_id,geometry,epsilon_buffer,width_buffer,azimuth
npartitions=1,,,,,
,int64,geometry,geometry,geometry,string
,...,...,...,...,...


In [19]:
print("Unique block_ids in spatial join:", buildings_blocks['block_id'].nunique())
print("Total rows in spatial join:", len(buildings_blocks))

Unique block_ids in spatial join: <dask_expr.expr.Scalar: expr=(DropDuplicates(frame=(FromGraph(8050434)[['block_id', 'geometry', 'epsilon_buffer', 'width_buffer', 'azimuth']])['block_id'], split_every=False, shuffle_method='p2p')).count(), dtype=int64>


/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/distributed/client.py:3370: UserWarning: Sending large graph of size 422.07 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:



clipped_buildings_area_to_buffer_ratio = epsilon_buffer_ratios / width_buffer_ratios

clipped_buildings_area_to_buffer_ratio = clipped_buildings_area_to_buffer_ratio.replace([np.inf, -np.inf], np.nan).fillna(0)

# Convert the ratio to a DataFrame (if it's not already)
ratio_df = clipped_buildings_area_to_buffer_ratio.to_frame(name='m8')

# Merge it into the blocks GeoDataFrame using the block_id index.
#blocks_with_m8 = blocks.merge(ratio_df, left_on='block_id', right_index=True, how='left')

In [ ]:
2.393200e+04

In [ ]:
clipped_buildings_area_to_buffer_ratio.describe()

In [ ]:
#blocks_with_m8 = blocks_with_m8.merge(gpd.GeoDataFrame(width_buffer_ratios,columns=['width_buffer_ratio']),left_on='block_id',right_index=True)
blocks_with_m8.to_parquet('blocks_with_width_buffer_ratio.geoparquet')

In [ ]:
blocks_with_m8['width_buffer_ratio'].describe()

In [ ]:
city_name = 'Nairobi'
blocks_with_m8 = results[0].set_geometry('geometry')
blocks_with_m8.to_parquet(f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}_with_m8.geoparquet')

In [15]:
client.close()